<h1> Criação da base de dados de empresas </h1>

In [1]:

import pandas as pd
import os
import numpy as np


<h2>Dados para Apoio <h2>

In [2]:
columns = ['cnae_princ_cod','cnae_text']
cnaes = pd.read_csv('./files/dados_apoio/F.K03200$Z.D50419.CNAECSV', sep=';', encoding='latin1', header=None, names=columns, index_col='cnae_princ_cod')
cnaes_dict = cnaes['cnae_text'].to_dict()


In [3]:
columns = ['municipio_code','municipio_nome']
municipios = pd.read_csv('./files/dados_apoio/F.K03200$Z.D50419.MUNICCSV', sep=';', encoding='latin1', header=None, names=columns, index_col='municipio_code')
municipios_dict = municipios['municipio_nome'].to_dict()


<h2>Estabelecimentos</h2> 

In [4]:
estabelecimentos_paths = os.listdir('./files/estabelecimentos')
columns = ['cnpj_basico','cnpj_ordem','cnpj_dv','id_matriz/filial','nome_fantasia','situacao_cadastral','data_situacao_cadastral',
           'motivo_situacao_cadastral','cidade_exterior','pais','data_inicio_atividade','cnae_princ_cod','cnaes_secundarios','tipo_logradouro',
           'logradouro','numero','complemento','bairro','cep','uf','municipio','ddd1','telefone1','ddd2','telefone2','ddd_fax','fax','e-mail','situacao_especial',
            'dt_situacao_especial']

In [5]:
estabelecimento_df = pd.read_csv(f'./files/estabelecimentos/{estabelecimentos_paths[0]}', sep=';', encoding='latin1', header=None, names=columns, on_bad_lines='skip')

C:\Users\ddemico\AppData\Local\Temp\ipykernel_8588\2583145129.py:1: DtypeWarning: Columns (8,21,22,24,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  estabelecimento_df = pd.read_csv(f'./files/estabelecimentos/{estabelecimentos_paths[0]}', sep=';', encoding='latin1', header=None, names=columns, on_bad_lines='skip')


In [6]:
#Tratamento Geral nos cnaes
cnaes_target = [4661300, 331411, 4614100, 4692300, 4789099, 2833000, 2541100, 0o161001, 0o161002, 4683400]
estabelecimento_cnae_filter = estabelecimento_df.loc[estabelecimento_df['cnae_princ_cod'].isin(cnaes_target)]
estabelecimento_remove_name = estabelecimento_cnae_filter.dropna(subset='nome_fantasia')
estabelecimento_remove_name['cnae_princ_text'] = estabelecimento_remove_name['cnae_princ_cod'].map(cnaes_dict)
 
def mapeia_cnae(cnae):
    if isinstance(cnae,float):
        return 'nan'
    
    mapeados = []
    for i in cnae.split(','):

        mapeados.append(cnaes_dict[int(i)])
    
    return ', '.join(mapeados)

estabelecimento_remove_name['cnae_secundario_text'] = estabelecimento_remove_name['cnaes_secundarios'].apply(mapeia_cnae)
estabelecimento = estabelecimento_remove_name


C:\Users\ddemico\AppData\Local\Temp\ipykernel_8588\1549135336.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estabelecimento_remove_name['cnae_princ_text'] = estabelecimento_remove_name['cnae_princ_cod'].map(cnaes_dict)
C:\Users\ddemico\AppData\Local\Temp\ipykernel_8588\1549135336.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estabelecimento_remove_name['cnae_secundario_text'] = estabelecimento_remove_name['cnaes_secundarios'].apply(mapeia_cnae)


In [7]:
# Tratamento Municipio
estabelecimento['municipio_name'] = estabelecimento['municipio'].map(municipios_dict)
estabelecimento['municipio_name'].head()

C:\Users\ddemico\AppData\Local\Temp\ipykernel_8588\3308801093.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estabelecimento['municipio_name'] = estabelecimento['municipio'].map(municipios_dict)


577    SANTA LUZIA DO PARUA
684     PRESIDENTE PRUDENTE
701                LONDRINA
731       SAO JOSE DO NORTE
837                PARACATU
Name: municipio_name, dtype: object

In [ ]:
def cnpj_completo(row):
    cnpj_basico = str(row['cnpj_basico'])
    cnpj_basico = cnpj_basico.zfill(8)
    cnpj_b_aj = f'{cnpj_basico[:2]}.{cnpj_basico[2:5]}.{cnpj_basico[5:]}'
    return f'{cnpj_b_aj}/000{row['cnpj_ordem']}-{row['cnpj_dv']}'
estabelecimento['cnpj_completo'] = estabelecimento.apply(cnpj_completo,axis=1)
estabelecimento.head()


C:\Users\ddemico\AppData\Local\Temp\ipykernel_8588\3877525789.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estabelecimento['cnpj_completo'] = estabelecimento.apply(cnpj_completo,axis=1)


,cnpj_basico,cnpj_ordem,cnpj_dv,id_matriz/filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,cidade_exterior,pais,...,telefone2,ddd_fax,fax,e-mail,situacao_especial,dt_situacao_especial,cnae_princ_text,cnae_secundario_text,municipio_name,cnpj_completo
577,4939944,1,80,1,CASA DA AGRICULTURA,2,20051103,0,NaN,NaN,...,NaN,NaN,NaN,EXATACONTABIL.MA@HOTMAIL.COM,NaN,NaN,Comércio varejista de outros produtos não espe...,Representantes comerciais e agentes do comérci...,SANTA LUZIA DO PARUA,49.399.44/0001-80
684,3300229,1,30,1,RACOES NEGRI,4,20181226,63,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,PRESIDENTE PRUDENTE,33.002.29/0001-30
701,3300275,1,39,1,DRY BALOES,8,20050427,1,NaN,NaN,...,NaN,43.0,3361115,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,LONDRINA,33.002.75/0001-39
731,92483320,1,68,1,A J CORREARIA E ESTOFARIA,8,20001213,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,SAO JOSE DO NORTE,92.483.320/0001-68
837,4940717,1,74,1,FERTIPLANT COMERCIO DE FERTILIZANTES,4,20190124,63,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"Comércio atacadista de defensivos agrícolas, a...","Transporte rodoviário de carga, exceto produto...",PARACATU,49.407.17/0001-74


In [9]:
def identificator_matriz_code_to_text(code) -> str:
    if code == 0o1:
        return 'MATRIZ'
    elif code == 0o2:
        return 'FILIAL'
    else:
        return 'NÃO IDENTIFICADO'

estabelecimento['matriz ou filial'] = estabelecimento['id_matriz/filial'].apply(identificator_matriz_code_to_text)
estabelecimento.head()

C:\Users\ddemico\AppData\Local\Temp\ipykernel_8588\1561042641.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estabelecimento['matriz ou filial'] = estabelecimento['id_matriz/filial'].apply(identificator_matriz_code_to_text)


,cnpj_basico,cnpj_ordem,cnpj_dv,id_matriz/filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,cidade_exterior,pais,...,ddd_fax,fax,e-mail,situacao_especial,dt_situacao_especial,cnae_princ_text,cnae_secundario_text,municipio_name,cnpj_completo,matriz ou filial
577,4939944,1,80,1,CASA DA AGRICULTURA,2,20051103,0,NaN,NaN,...,NaN,NaN,EXATACONTABIL.MA@HOTMAIL.COM,NaN,NaN,Comércio varejista de outros produtos não espe...,Representantes comerciais e agentes do comérci...,SANTA LUZIA DO PARUA,49.399.44/0001-80,MATRIZ
684,3300229,1,30,1,RACOES NEGRI,4,20181226,63,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,PRESIDENTE PRUDENTE,33.002.29/0001-30,MATRIZ
701,3300275,1,39,1,DRY BALOES,8,20050427,1,NaN,NaN,...,43.0,3361115,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,LONDRINA,33.002.75/0001-39,MATRIZ
731,92483320,1,68,1,A J CORREARIA E ESTOFARIA,8,20001213,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,SAO JOSE DO NORTE,92.483.320/0001-68,MATRIZ
837,4940717,1,74,1,FERTIPLANT COMERCIO DE FERTILIZANTES,4,20190124,63,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"Comércio atacadista de defensivos agrícolas, a...","Transporte rodoviário de carga, exceto produto...",PARACATU,49.407.17/0001-74,MATRIZ


In [10]:
# Retira todas as empresas inativas
def identificator_sit_cadastral(code) -> str:
    if code == 0o1:
        return 'NULA'
    elif code == 0o2:
        return 'ATIVA'
    if code == 0o3:
        return 'SUSPENSA'
    elif code == 0o4:
        return 'INAPTA'
    elif code == 8:
        return 'BAIXADA'
    else:
        return 'NÃO IDENTIFICADO'

estabelecimento = estabelecimento.loc[estabelecimento['situacao_cadastral'] == 0o2]
estabelecimento

,cnpj_basico,cnpj_ordem,cnpj_dv,id_matriz/filial,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,cidade_exterior,pais,...,ddd_fax,fax,e-mail,situacao_especial,dt_situacao_especial,cnae_princ_text,cnae_secundario_text,municipio_name,cnpj_completo,matriz ou filial
577,4939944,1,80,1,CASA DA AGRICULTURA,2,20051103,0,NaN,NaN,...,NaN,NaN,EXATACONTABIL.MA@HOTMAIL.COM,NaN,NaN,Comércio varejista de outros produtos não espe...,Representantes comerciais e agentes do comérci...,SANTA LUZIA DO PARUA,49.399.44/0001-80,MATRIZ
2605,5429,1,71,1,PRISMA PATETEIRAS HIDRAULICAS,2,20051103,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,Manutenção e reparação de equipamentos hidrául...,SAO PAULO,54.29./0001-71,MATRIZ
3964,92570746,1,59,1,LOJA DELA RO,2,20191029,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,ARROIO GRANDE,92.570.746/0001-59,MATRIZ
4078,1660966,1,18,1,BAZAR ELZA,2,20240103,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,SAO PAULO,16.609.66/0001-18,MATRIZ
4966,11627,1,48,1,GARAPOS,2,20041218,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,GOIANIA,11.627./0001-48,MATRIZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4742268,29682798,2,61,2,INOVATEC PECAS E MAQUINAS AGRICOLAS,2,20190429,0,NaN,NaN,...,NaN,NaN,WAGNER.MILET@LOJAINOVAGRI.COM.BR,NaN,NaN,"Comércio atacadista de mercadorias em geral, c...",Representantes comerciais e agentes do comérci...,PARIPIRANGA,29.682.798/0002-61,FILIAL
4743736,34019922,1,62,1,SUPERCOMAM,2,20190625,0,NaN,NaN,...,NaN,NaN,COMAM.ANAJAS@SUPERCOMAM.COM.BR,NaN,NaN,Comércio varejista de outros produtos não espe...,Manutenção e reparação de equipamentos e produ...,ANAJAS,34.019.922/0001-62,MATRIZ
4747722,35356730,1,4,1,MAQFAMA,2,20230313,0,NaN,NaN,...,NaN,NaN,MAQFAMA@OUTLOOK.COM,NaN,NaN,Comércio varejista de outros produtos não espe...,"Comércio varejista de móveis, Comércio varejis...",SAO LUIS,35.356.730/0001-4,MATRIZ
4748686,35663253,1,20,1,NASTRI SERVICE,2,20191129,0,NaN,NaN,...,NaN,NaN,LINS.DOCUMENTOS@GMAIL.COM,NaN,NaN,Representantes comerciais e agentes do comérci...,Consultoria em tecnologia da informação,SANTANA DE PARNAIBA,35.663.253/0001-20,MATRIZ


In [11]:
colunas_remove = ['cnpj_ordem','cnpj_dv','id_matriz/filial','situacao_cadastral','data_situacao_cadastral',
                  'motivo_situacao_cadastral','cidade_exterior','ddd_fax','fax','situacao_especial','dt_situacao_especial','ddd2','telefone2']
estabelecimento = estabelecimento.drop(columns=colunas_remove)
estabelecimento


,cnpj_basico,nome_fantasia,pais,data_inicio_atividade,cnae_princ_cod,cnaes_secundarios,tipo_logradouro,logradouro,numero,complemento,...,uf,municipio,ddd1,telefone1,e-mail,cnae_princ_text,cnae_secundario_text,municipio_name,cnpj_completo,matriz ou filial
577,4939944,CASA DA AGRICULTURA,NaN,20020313,4789099,"4619200,4692300,4744001,4744099,4771704",AVENIDA,AVENIDA PROFESSOR JOAO MORAIS DE SOUSA,548,NaN,...,MA,1285,98.0,33741352,EXATACONTABIL.MA@HOTMAIL.COM,Comércio varejista de outros produtos não espe...,Representantes comerciais e agentes do comérci...,SANTA LUZIA DO PARUA,49.399.44/0001-80,MATRIZ
2605,5429,PRISMA PATETEIRAS HIDRAULICAS,NaN,19940517,4789099,3314702,RUA,"JOAQUIM ANTONIO ALVES,",236,NaN,...,SP,7107,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,Manutenção e reparação de equipamentos hidrául...,SAO PAULO,54.29./0001-71,MATRIZ
3964,92570746,LOJA DELA RO,NaN,19890131,4789099,NaN,RUA,AIMORE SOARES CARRICOND,SN,NaN,...,RS,8525,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,ARROIO GRANDE,92.570.746/0001-59,MATRIZ
4078,1660966,BAZAR ELZA,NaN,19970219,4789099,NaN,RUA,ANTONIO MARCONDES,699,NaN,...,SP,7107,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,SAO PAULO,16.609.66/0001-18,MATRIZ
4966,11627,GARAPOS,NaN,19811201,4789099,NaN,RUA,MOISES SANTANA,34,NaN,...,GO,9373,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,GOIANIA,11.627./0001-48,MATRIZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4742268,29682798,INOVATEC PECAS E MAQUINAS AGRICOLAS,NaN,20190429,4692300,"4612500,4614100,4623106,4623199,4632001,466130...",RUA,MANOEL COELHO CRUZ,234 A,NaN,...,BA,3777,79.0,98422410.0,WAGNER.MILET@LOJAINOVAGRI.COM.BR,"Comércio atacadista de mercadorias em geral, c...",Representantes comerciais e agentes do comérci...,PARIPIRANGA,29.682.798/0002-61,FILIAL
4743736,34019922,SUPERCOMAM,NaN,20190625,4789099,"3319800,4530705,4732600,4741500,4742300,474400...",AVENIDA,BARAO DO RIO BRANCO,195,NaN,...,PA,413,93.0,91249282.0,COMAM.ANAJAS@SUPERCOMAM.COM.BR,Comércio varejista de outros produtos não espe...,Manutenção e reparação de equipamentos e produ...,ANAJAS,34.019.922/0001-62,MATRIZ
4747722,35356730,MAQFAMA,NaN,20191030,4789099,"4754701,4763604,4789004",RUA,TERESINA,21,"CONJ RESIDENCIAL TIRADENTES, AV EVANDRO SILVA",...,MA,921,98.0,87371275.0,MAQFAMA@OUTLOOK.COM,Comércio varejista de outros produtos não espe...,"Comércio varejista de móveis, Comércio varejis...",SAO LUIS,35.356.730/0001-4,MATRIZ
4748686,35663253,NASTRI SERVICE,NaN,20191129,4614100,6204000,ALAMEDA,RENOIR,480,NaN,...,SP,7047,11.0,43264357.0,LINS.DOCUMENTOS@GMAIL.COM,Representantes comerciais e agentes do comérci...,Consultoria em tecnologia da informação,SANTANA DE PARNAIBA,35.663.253/0001-20,MATRIZ


In [12]:
def definir_idade(dt_nasc):
    vlr = str(dt_nasc)
    ano = vlr[:4]
    return 2025 - int(ano)

estabelecimento['idade'] = estabelecimento['data_inicio_atividade'].apply(definir_idade)
estabelecimento.head()

,cnpj_basico,nome_fantasia,pais,data_inicio_atividade,cnae_princ_cod,cnaes_secundarios,tipo_logradouro,logradouro,numero,complemento,...,municipio,ddd1,telefone1,e-mail,cnae_princ_text,cnae_secundario_text,municipio_name,cnpj_completo,matriz ou filial,idade
577,4939944,CASA DA AGRICULTURA,NaN,20020313,4789099,"4619200,4692300,4744001,4744099,4771704",AVENIDA,AVENIDA PROFESSOR JOAO MORAIS DE SOUSA,548,NaN,...,1285,98.0,33741352,EXATACONTABIL.MA@HOTMAIL.COM,Comércio varejista de outros produtos não espe...,Representantes comerciais e agentes do comérci...,SANTA LUZIA DO PARUA,49.399.44/0001-80,MATRIZ,23
2605,5429,PRISMA PATETEIRAS HIDRAULICAS,NaN,19940517,4789099,3314702,RUA,"JOAQUIM ANTONIO ALVES,",236,NaN,...,7107,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,Manutenção e reparação de equipamentos hidrául...,SAO PAULO,54.29./0001-71,MATRIZ,31
3964,92570746,LOJA DELA RO,NaN,19890131,4789099,NaN,RUA,AIMORE SOARES CARRICOND,SN,NaN,...,8525,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,ARROIO GRANDE,92.570.746/0001-59,MATRIZ,36
4078,1660966,BAZAR ELZA,NaN,19970219,4789099,NaN,RUA,ANTONIO MARCONDES,699,NaN,...,7107,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,SAO PAULO,16.609.66/0001-18,MATRIZ,28
4966,11627,GARAPOS,NaN,19811201,4789099,NaN,RUA,MOISES SANTANA,34,NaN,...,9373,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,GOIANIA,11.627./0001-48,MATRIZ,44


In [13]:
estabelecimento['n_cnaes_secundarios'] = estabelecimento['cnaes_secundarios'].apply(
    lambda x: len(str(x).split(',')) if pd.notnull(x) else 0
)
estabelecimento.head()

,cnpj_basico,nome_fantasia,pais,data_inicio_atividade,cnae_princ_cod,cnaes_secundarios,tipo_logradouro,logradouro,numero,complemento,...,ddd1,telefone1,e-mail,cnae_princ_text,cnae_secundario_text,municipio_name,cnpj_completo,matriz ou filial,idade,n_cnaes_secundarios
577,4939944,CASA DA AGRICULTURA,NaN,20020313,4789099,"4619200,4692300,4744001,4744099,4771704",AVENIDA,AVENIDA PROFESSOR JOAO MORAIS DE SOUSA,548,NaN,...,98.0,33741352,EXATACONTABIL.MA@HOTMAIL.COM,Comércio varejista de outros produtos não espe...,Representantes comerciais e agentes do comérci...,SANTA LUZIA DO PARUA,49.399.44/0001-80,MATRIZ,23,5
2605,5429,PRISMA PATETEIRAS HIDRAULICAS,NaN,19940517,4789099,3314702,RUA,"JOAQUIM ANTONIO ALVES,",236,NaN,...,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,Manutenção e reparação de equipamentos hidrául...,SAO PAULO,54.29./0001-71,MATRIZ,31,1
3964,92570746,LOJA DELA RO,NaN,19890131,4789099,NaN,RUA,AIMORE SOARES CARRICOND,SN,NaN,...,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,ARROIO GRANDE,92.570.746/0001-59,MATRIZ,36,0
4078,1660966,BAZAR ELZA,NaN,19970219,4789099,NaN,RUA,ANTONIO MARCONDES,699,NaN,...,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,SAO PAULO,16.609.66/0001-18,MATRIZ,28,0
4966,11627,GARAPOS,NaN,19811201,4789099,NaN,RUA,MOISES SANTANA,34,NaN,...,NaN,NaN,NaN,Comércio varejista de outros produtos não espe...,nan,GOIANIA,11.627./0001-48,MATRIZ,44,0


<h2>Empresas</h2>

In [14]:
empresas_paths = os.listdir('./files/empresas')
columns = ['cnpj_basico','razão_social','natureza_juridica','qualificacao_responsavel','capital_social','porte_empresa','ente_federativo']

In [15]:
empresas = []
for p in empresas_paths: 
    empresa_df_temp = pd.read_csv(f'./files/empresas/{p}',sep=';',encoding='latin1', header=None, names=columns)
    empresa_df_temp = empresa_df_temp.drop(columns=['natureza_juridica','qualificacao_responsavel','ente_federativo'])
    empresas.append(empresa_df_temp)
    
empresa_df = pd.concat(empresas)

In [16]:
def code_empresa_to_text(code:int) -> str:
    if code == 00:
        return 'NÃO INFORMADO'
    elif code == 0o1:
        return 'MICRO EMPRESA'
    elif code == 0o3:
        return 'EMPRESA DE PEQUENO PORTE'
    elif code == 0o5:
        return 'DEMAIS'
    else: 
        return 'CÓDIGO NÃO IDENTIFICADO'

empresa_df['porte_empresa_text'] = empresa_df['porte_empresa'].apply(code_empresa_to_text)
empresa_df.head()
    

,cnpj_basico,razão_social,capital_social,porte_empresa,porte_empresa_text
0,41273588,STUDIO MVO2 LTDA,"20000,00",1.0,MICRO EMPRESA
1,41273589,IRENILDA OLIVEIRA SILVA 11338767810,"5000,00",1.0,MICRO EMPRESA
2,41273590,MARIA DAS MERCES SOARES LEMOS,"0,00",5.0,DEMAIS
3,41273591,CASSIO APARECIDO LOPES 06457523803,"1000,00",1.0,MICRO EMPRESA
4,41273592,41.273.592 HELIO DE JESUS PEREIRA,"30000,00",1.0,MICRO EMPRESA


In [17]:
def receita_porte(code):
    if code == 00:
        return np.nan
    elif code == 0o1:
        return '0 - 360.000.000,00'
    elif code == 0o3:
        return '360.000.000,01 - 4.800.000,00'
    elif code == 0o5:
        return '4.800.000.000,01 - '
    else: 
        return np.nan

    
empresa_df['receita_estimada_por_porte'] = empresa_df['porte_empresa'].apply(receita_porte)
empresa_df.head()

,cnpj_basico,razão_social,capital_social,porte_empresa,porte_empresa_text,receita_estimada_por_porte
0,41273588,STUDIO MVO2 LTDA,"20000,00",1.0,MICRO EMPRESA,"0 - 360.000.000,00"
1,41273589,IRENILDA OLIVEIRA SILVA 11338767810,"5000,00",1.0,MICRO EMPRESA,"0 - 360.000.000,00"
2,41273590,MARIA DAS MERCES SOARES LEMOS,"0,00",5.0,DEMAIS,"4.800.000.000,01 -"
3,41273591,CASSIO APARECIDO LOPES 06457523803,"1000,00",1.0,MICRO EMPRESA,"0 - 360.000.000,00"
4,41273592,41.273.592 HELIO DE JESUS PEREIRA,"30000,00",1.0,MICRO EMPRESA,"0 - 360.000.000,00"


In [18]:
def func_por_porte(code):
    if code == 00:
        return np.nan
    elif code == 0o1:
        return '1 - 9'
    elif code == 0o3:
        return '10 - 49'
    elif code == 0o5:
        return '50 - '
    else: 
        return np.nan

empresa_df['func_estimada_por_porte'] = empresa_df['porte_empresa'].apply(func_por_porte)
empresa_df.head()

,cnpj_basico,razão_social,capital_social,porte_empresa,porte_empresa_text,receita_estimada_por_porte,func_estimada_por_porte
0,41273588,STUDIO MVO2 LTDA,"20000,00",1.0,MICRO EMPRESA,"0 - 360.000.000,00",1 - 9
1,41273589,IRENILDA OLIVEIRA SILVA 11338767810,"5000,00",1.0,MICRO EMPRESA,"0 - 360.000.000,00",1 - 9
2,41273590,MARIA DAS MERCES SOARES LEMOS,"0,00",5.0,DEMAIS,"4.800.000.000,01 -",50 -
3,41273591,CASSIO APARECIDO LOPES 06457523803,"1000,00",1.0,MICRO EMPRESA,"0 - 360.000.000,00",1 - 9
4,41273592,41.273.592 HELIO DE JESUS PEREIRA,"30000,00",1.0,MICRO EMPRESA,"0 - 360.000.000,00",1 - 9


In [19]:
df_empresa_estabelecimento = pd.merge(estabelecimento, empresa_df, on='cnpj_basico', how='left')
df_empresa_estabelecimento.head()

,cnpj_basico,nome_fantasia,pais,data_inicio_atividade,cnae_princ_cod,cnaes_secundarios,tipo_logradouro,logradouro,numero,complemento,...,cnpj_completo,matriz ou filial,idade,n_cnaes_secundarios,razão_social,capital_social,porte_empresa,porte_empresa_text,receita_estimada_por_porte,func_estimada_por_porte
0,4939944,CASA DA AGRICULTURA,NaN,20020313,4789099,"4619200,4692300,4744001,4744099,4771704",AVENIDA,AVENIDA PROFESSOR JOAO MORAIS DE SOUSA,548,NaN,...,49.399.44/0001-80,MATRIZ,23,5,F.BARBOSA DE SOUSA,"10000,00",1.0,MICRO EMPRESA,"0 - 360.000.000,00",1 - 9
1,5429,PRISMA PATETEIRAS HIDRAULICAS,NaN,19940517,4789099,3314702,RUA,"JOAQUIM ANTONIO ALVES,",236,NaN,...,54.29./0001-71,MATRIZ,31,1,M N PEREIRA HIDRAULICA,"0,00",1.0,MICRO EMPRESA,"0 - 360.000.000,00",1 - 9
2,92570746,LOJA DELA RO,NaN,19890131,4789099,NaN,RUA,AIMORE SOARES CARRICOND,SN,NaN,...,92.570.746/0001-59,MATRIZ,36,0,OTAVIANO ANTONIO CARDOZO,"0,00",1.0,MICRO EMPRESA,"0 - 360.000.000,00",1 - 9
3,1660966,BAZAR ELZA,NaN,19970219,4789099,NaN,RUA,ANTONIO MARCONDES,699,NaN,...,16.609.66/0001-18,MATRIZ,28,0,ELZA SANTOS SOBRAL-PRESENTES,"0,00",1.0,MICRO EMPRESA,"0 - 360.000.000,00",1 - 9
4,11627,GARAPOS,NaN,19811201,4789099,NaN,RUA,MOISES SANTANA,34,NaN,...,11.627./0001-48,MATRIZ,44,0,GARAPOS MANUTENCAO EM POSTOS LTDA,"0,00",1.0,MICRO EMPRESA,"0 - 360.000.000,00",1 - 9


In [27]:
def retire(x):
    x = str(x)
    x = x.replace(',','.')
    return x 
df_empresa_estabelecimento['capital_social'] = df_empresa_estabelecimento['capital_social'].apply(retire)

In [28]:
df_empresa_estabelecimento.head()


,cnpj_basico,nome_fantasia,pais,data_inicio_atividade,cnae_princ_cod,cnaes_secundarios,tipo_logradouro,logradouro,numero,complemento,...,cnpj_completo,matriz ou filial,idade,n_cnaes_secundarios,razão_social,capital_social,porte_empresa,porte_empresa_text,receita_estimada_por_porte,func_estimada_por_porte
0,4939944,CASA DA AGRICULTURA,NaN,20020313,4789099,"4619200,4692300,4744001,4744099,4771704",AVENIDA,AVENIDA PROFESSOR JOAO MORAIS DE SOUSA,548,NaN,...,49.399.44/0001-80,MATRIZ,23,5,F.BARBOSA DE SOUSA,1000000,1.0,MICRO EMPRESA,"0 - 360.000.000,00",1 - 9
1,5429,PRISMA PATETEIRAS HIDRAULICAS,NaN,19940517,4789099,3314702,RUA,"JOAQUIM ANTONIO ALVES,",236,NaN,...,54.29./0001-71,MATRIZ,31,1,M N PEREIRA HIDRAULICA,000,1.0,MICRO EMPRESA,"0 - 360.000.000,00",1 - 9
2,92570746,LOJA DELA RO,NaN,19890131,4789099,NaN,RUA,AIMORE SOARES CARRICOND,SN,NaN,...,92.570.746/0001-59,MATRIZ,36,0,OTAVIANO ANTONIO CARDOZO,000,1.0,MICRO EMPRESA,"0 - 360.000.000,00",1 - 9
3,1660966,BAZAR ELZA,NaN,19970219,4789099,NaN,RUA,ANTONIO MARCONDES,699,NaN,...,16.609.66/0001-18,MATRIZ,28,0,ELZA SANTOS SOBRAL-PRESENTES,000,1.0,MICRO EMPRESA,"0 - 360.000.000,00",1 - 9
4,11627,GARAPOS,NaN,19811201,4789099,NaN,RUA,MOISES SANTANA,34,NaN,...,11.627./0001-48,MATRIZ,44,0,GARAPOS MANUTENCAO EM POSTOS LTDA,000,1.0,MICRO EMPRESA,"0 - 360.000.000,00",1 - 9


<h2> Modelo de Predicão de Receita </h2>

In [26]:
df_empresa_estabelecimento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5452 entries, 0 to 5451
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   cnpj_basico                 5452 non-null   int64  
 1   nome_fantasia               5452 non-null   object 
 2   pais                        7 non-null      float64
 3   data_inicio_atividade       5452 non-null   int64  
 4   cnae_princ_cod              5452 non-null   int64  
 5   cnaes_secundarios           4050 non-null   object 
 6   tipo_logradouro             5416 non-null   object 
 7   logradouro                  5452 non-null   object 
 8   numero                      5452 non-null   object 
 9   complemento                 2365 non-null   object 
 10  bairro                      5449 non-null   object 
 11  cep                         5447 non-null   float64
 12  uf                          5452 non-null   object 
 13  municipio                   5452 

In [78]:
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.impute import SimpleImputer


In [79]:
def parse_revenue_range_and_target(range_str):
    min_val, max_val, target_val = np.nan, np.nan, np.nan

    if isinstance(range_str, str):
        cleaned_str = range_str.strip()
        parts = cleaned_str.split('-')

        if len(parts) == 2:
            try:
                min_val_str = parts[0].strip()
                max_val_str = parts[1].strip()

                # Limpa formato numérico brasileiro (remove ., troca , por .)
                min_val = float(min_val_str.replace('.', '').replace(',', '.'))

                if max_val_str == '':
                    # Range aberto (ex: "X - ") -> Usar o mínimo como target proxy
                    max_val = np.inf # Representa o limite superior como infinito
                    target_val = min_val # Target proxy é o limite inferior
                else:
                    # Range fechado (ex: "MIN - MAX") -> Usar o ponto médio como target proxy
                    max_val = float(max_val_str.replace('.', '').replace(',', '.'))
                    target_val = (min_val + max_val) / 2

            except ValueError:
                pass # Permanece NaN se a conversão falhar
        # Adicionar tratamento para outros formatos se existirem (ex: 'Acima de X')
        # Por agora, eles resultarão em NaN
    return min_val, max_val, target_val

# Aplica a função e cria colunas temporárias
results = df_empresa_estabelecimento['receita_estimada_por_porte'].apply(parse_revenue_range_and_target)

# Cria as colunas separadas a partir dos resultados
df_empresa_estabelecimento['faturamento_min'] = [x[0] for x in results]
df_empresa_estabelecimento['faturamento_max'] = [x[1] for x in results]
df_empresa_estabelecimento['faturamento_proxy_target'] = [x[2] for x in results]
df_empresa_estabelecimento.dropna(subset=['faturamento_proxy_target'], inplace=True)
print(f"\nDataFrame após parsear faturamento e criar target proxy. Linhas restantes: {len(df_empresa_estabelecimento)}")
print("Exemplo de ranges parseados:")
print(df_empresa_estabelecimento[['receita_estimada_por_porte', 'faturamento_min', 'faturamento_max', 'faturamento_proxy_target']].head())



DataFrame após parsear faturamento e criar target proxy. Linhas restantes: 3628
Exemplo de ranges parseados:
  receita_estimada_por_porte  faturamento_min  faturamento_max  \
0         0 - 360.000.000,00              0.0      360000000.0   
1         0 - 360.000.000,00              0.0      360000000.0   
2         0 - 360.000.000,00              0.0      360000000.0   
3         0 - 360.000.000,00              0.0      360000000.0   
4         0 - 360.000.000,00              0.0      360000000.0   

   faturamento_proxy_target  
0               180000000.0  
1               180000000.0  
2               180000000.0  
3               180000000.0  
4               180000000.0  


In [80]:
features = [
    'capital_social',
    'idade',
    'cnae_princ_cod',
    'n_cnaes_secundarios',
    'matriz ou filial',
    'porte_empresa'
]

X = df_empresa_estabelecimento[features]
y = df_empresa_estabelecimento['faturamento_proxy_target']


In [81]:
numerical_features = ['capital_social','idade','n_cnaes_secundarios']
categorical_features = ['cnae_princ_cod','matriz ou filial']

def log1p_transform(X):
    return np.log1p(X)
log1p_transformer = FunctionTransformer(log1p_transform)


numerical_transformers = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('log1p',log1p_transformer),
    ('scaler',StandardScaler())
])

categorical_transformers = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers = [
        ('num',numerical_transformers,numerical_features),
        ('cat',categorical_transformers,categorical_features)
    ],remainder='passthrough'
)


In [82]:
model_pipeline = Pipeline(steps=[('preprocessor',preprocessor),
                                 ('regressor', RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1, max_depth=10))])

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=42)
print(f"Tamanho do conjunto de treino: {len(X_train)}")
print(f"Tamanho do conjunto de teste: {len(X_test)}")


Tamanho do conjunto de treino: 2902
Tamanho do conjunto de teste: 726


In [84]:
model_pipeline.fit(X_train,y_train)

c:\Users\ddemico\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('log1p',
                                                                   FunctionTransformer(func=<function log1p_transform at 0x0000017B0599C5E0>)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['capital_social', 'idade',
                                                   'n_cnaes_secundarios']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['cnae_princ_cod',
                                                   'matriz ou filial'])])),
                ('regressor',
                 RandomForestRegressor(max_depth=10, n_estimators=200,
                                       n_jobs=-1, random_state=42))])

In [85]:
y_pred = model_pipeline.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE) na previsão do proxy: {mae:,.2f}")
print(f"Root Mean Squared Error (RMSE) na previsão do proxy: {rmse:,.2f}")
print(f"R-squared na previsão do proxy: {r2:.4f}")


Mean Absolute Error (MAE) na previsão do proxy: 0.00
Root Mean Squared Error (RMSE) na previsão do proxy: 0.00
R-squared na previsão do proxy: 1.0000


In [86]:
print("\nGerando estimativas de faturamento para todo o DataFrame original...")
df_empresa_estabelecimento['faturamento_estimado'] = model_pipeline.predict(X)


Gerando estimativas de faturamento para todo o DataFrame original...


In [87]:
df_empresa_estabelecimento['faturamento_estimado'] = df_empresa_estabelecimento['faturamento_estimado'].round(2)

In [88]:
print(df_empresa_estabelecimento[['cnpj_completo', 'porte_empresa_text', 'receita_estimada_por_porte', 'faturamento_proxy_target', 'faturamento_estimado',
           'capital_social', 'idade', 'n_cnaes_secundarios']])

df_empresa_estabelecimento.to_excel('./files/dados_feitos/teste_c_modelo.xlsx')

           cnpj_completo porte_empresa_text receita_estimada_por_porte  \
0      49.399.44/0001-80      MICRO EMPRESA         0 - 360.000.000,00   
1         54.29./0001-71      MICRO EMPRESA         0 - 360.000.000,00   
2     92.570.746/0001-59      MICRO EMPRESA         0 - 360.000.000,00   
3      16.609.66/0001-18      MICRO EMPRESA         0 - 360.000.000,00   
4        11.627./0001-48      MICRO EMPRESA         0 - 360.000.000,00   
...                  ...                ...                        ...   
5440  41.906.712/0001-51      MICRO EMPRESA         0 - 360.000.000,00   
5441  41.913.348/0001-57      MICRO EMPRESA         0 - 360.000.000,00   
5442  41.922.982/0001-56      MICRO EMPRESA         0 - 360.000.000,00   
5443  41.926.166/0001-10      MICRO EMPRESA         0 - 360.000.000,00   
5445  41.936.172/0001-59      MICRO EMPRESA         0 - 360.000.000,00   

      faturamento_proxy_target  faturamento_estimado capital_social  idade  \
0                  180000000.0   

PermissionError: [Errno 13] Permission denied: './files/dados_feitos/teste_c_modelo.xlsx'